In [98]:
import pandas as pd
from datetime import datetime as dt
from datetime import timedelta
import matplotlib.pyplot as plt
import seaborn as sns
from statistics import mode

In [95]:
file_order = 0

filename = [ "202101-divvy-tripdata.csv",
            "202102-divvy-tripdata.csv",
            "202103-divvy-tripdata.csv",
            "202104-divvy-tripdata.csv",
            "202105-divvy-tripdata.csv",
            "202106-divvy-tripdata.csv",
            "202107-divvy-tripdata.csv",
            "202108-divvy-tripdata.csv",
            "202109-divvy-tripdata.csv",
            "202110-divvy-tripdata.csv",
            "202111-divvy-tripdata.csv",
            "202112-divvy-tripdata.csv",
            "202201-divvy-tripdata.csv",
            "202202-divvy-tripdata.csv",
            "202203-divvy-tripdata.csv",
            "202204-divvy-tripdata.csv",
            "202205-divvy-tripdata.csv",
            "202206-divvy-tripdata.csv",
            "202207-divvy-tripdata.csv",
            "202208-divvy-tripdata.csv",
            "202209-divvy-tripdata.csv",
            "202210-divvy-tripdata.csv",
            "202211-divvy-tripdata.csv"]

after = ['ride_id',
         'rideable_type',
         'started_at', 
         'ended_at',         
         'ride_length',
         'day_of_week',
         'start_station_name',
         'start_station_id',                  
         'end_station_name',                          
         'end_station_id',
         'start_lat',
         'start_lng',
         'end_lat',
         'end_lng',
         'user_type']

format_datetime1 = "%Y-%m-%d %H:%M:%S"
format_datetime2 = "%m/%d/%Y %H:%M"

avg_ride_length_casual = []
avg_ride_length_member = []
qtty_rides_casual = []
qtty_rides_member = []
negative_ride = pd.DataFrame()
ride_bigger_24h = pd.DataFrame()


In [96]:
for file_order in range(23):
    bike_df = pd.read_csv(f"../CaseStudy/TripData/{filename[file_order]}",low_memory = False)

    file_order += 1

    # Renaming columns in order to standardize and make sure data integrity.
    # bike_df.rename(columns = {list(bike_df.columns)[i]:after[i] for i in range(len(after))}, inplace = True)

    # Dropping unused data for analysis
    bike_df.drop(['rideable_type',
                  'start_lat',
                  'start_lng',
                  'end_lat',
                  'end_lng'], axis = 1, inplace = True)
       
    # Transform to Datetime data format
    bike_df["ended_at"] = pd.to_datetime(bike_df["ended_at"], format = format_datetime1)
    bike_df["started_at"] = pd.to_datetime(bike_df["started_at"], format = format_datetime1)
    
    # Make a new column based on 'ended_at' and 'started_at' column to ensure data precision.
    bike_df.insert(2,"ride_length",(bike_df["ended_at"] - bike_df["started_at"]).dt.total_seconds())
    
    #Separate and Remove rides that last more than 24h
    ride_bigger_24h = pd.concat([ride_bigger_24h, (bike_df.loc[bike_df['ride_length'] > 86400])], axis=0)
    bike_df.drop(list(bike_df.loc[bike_df['ride_length'] < 0].index.values), axis = 0, inplace = True)
    
    #Separate in different dataframe and remove rides which 'ended before started', improving data consistency
    negative_ride = pd.concat([negative_ride, (bike_df.loc[bike_df['ride_length'] < 0])], axis=0)
    bike_df.drop(list(bike_df.loc[bike_df['ride_length'] < 0].index.values), axis = 0, inplace = True)

    # Make a new column based on the 'started_at' to check which weekday the ride happenned
    bike_df.insert(3,"day_of_week",bike_df["started_at"].dt.weekday)
    bike_df.insert(3,"day",bike_df["started_at"].dt.day)
    bike_df.insert(3,"month",bike_df["started_at"].dt.month)
    bike_df.insert(3,"year",bike_df["started_at"].dt.year)
      
    bike_df.drop(['ride_id',
                  'started_at',                  
                  'ended_at',
                  'start_station_name',
                  'start_station_id', 
                  'end_station_name',
                  'end_station_id'], axis = 1, inplace = True)

    # Sort rides in the month in a chronologically ascending way
    # bike_df = bike_df.sort_values(by = "started_at")

    if file_order == 1:
        bike_complete = bike_df
    else:
        bike_complete = pd.concat([bike_complete, bike_df], axis=0)
    
bike_complete


,ride_length,year,month,day,day_of_week,member_casual
0,625.0,2021,1,23,5,member
1,244.0,2021,1,27,2,member
2,80.0,2021,1,21,3,member
3,702.0,2021,1,7,3,member
4,43.0,2021,1,23,5,casual
...,...,...,...,...,...,...
337730,718.0,2022,11,25,4,member
337731,2016.0,2022,11,22,1,casual
337732,568.0,2022,11,6,6,casual
337733,20148.0,2022,11,6,6,casual


In [97]:
#Save extra dataframes into CSV files

ride_bigger_24h.to_csv('ride_bigger_24h')
negative_ride.to_csv('Ended_before_start')

In [92]:
#Reset index para enumerate rows

bike_complete = bike_complete.reset_index()
bike_complete.drop(['index'], axis = 1, inplace=True)
bike_complete

,ride_length,year,month,day,day_of_week,member_casual
0,625.0,2021,1,23,5,member
1,244.0,2021,1,27,2,member
2,80.0,2021,1,21,3,member
3,702.0,2021,1,7,3,member
4,43.0,2021,1,23,5,casual
...,...,...,...,...,...,...
11080722,718.0,2022,11,25,4,member
11080723,2016.0,2022,11,22,1,casual
11080724,568.0,2022,11,6,6,casual
11080725,20148.0,2022,11,6,6,casual


In [ ]:
#Save the filtered data into a csv file

bike_complete.to_csv("filtered_bike_data3.csv")